In [6]:
import openai
import pandas as pd
##install and import praw lib to create a connection with reddit
import praw
import os

In [39]:
os.environ['OPENAI_API_KEY'] = ''

In [40]:
openai.api_key = os.getenv('OPENAI_API_KEY')

In [14]:
##Reddit credentials, create an app in reddit to get them

client_id = ''
client_secret = ''




In [16]:
reddit = praw.Reddit(client_id = client_id,
                     client_secret = client_secret, 
                     user_agent = 'sentiment analysis test'
                    )

In [18]:
for submission in reddit.subreddit("stocks").hot(limit=5):
    print(submission.title)

Rate My Portfolio - r/Stocks Quarterly Thread December 2023
/r/Stocks Weekend Discussion Saturday - Dec 02, 2023
Why is the market pricing in rate cutes when the fed has been clear they arent thinking about cuts?
Apple TV Plus and Paramount Plus reportedly discussing cheap bundle plan to take on Netflix
Uber, Jabil, Builders FirstSource to Join S&P 500


In [19]:
subreddit_stocks = reddit.subreddit("stocks")

In [21]:
for post in subreddit_stocks.hot(limit=5):
    print(post.title)
    submission = reddit.submission(post.id)
    ##print 2 commnets
    counter = 0
    for comment  in submission.comments:
        print(comment.body)
        counter+=1
        if counter == 2:
            break

Rate My Portfolio - r/Stocks Quarterly Thread December 2023
My current portfolio is 50% MTCH (MatchGroup) and 50% EZJ (EasyJet). Anyone have any suggestions for possible further diversification (looking to hold 4-8 stocks in total).
Portfolio - INMD, TAYD, TGTX, AMLX, PFE. (I hold 100 shares and sell occasionally on rips or sell covered calls interchangbly. If not, then I hold them long term until I feel like I make a decent profit and switch to CSP).

I hold NEP and will continue holding this as it is my biggest winner so far with a killer dividend. 

I plan to add PERI once i have the capital and add MOD once it has some healthy pullback.

I focus on quality, valuation, and how much more the stock can grow in market cap.
/r/Stocks Weekend Discussion Saturday - Dec 02, 2023
Why is the market pricing in rate cutes when the fed has been clear they arent thinking about cuts?
They saw JPOW wink when he said no rate cuts.
Of course the Fed is going to say that. Inflation is partly psycholo

In [30]:
def get_titles_and_comments(subreddit="stocks", limit=6, num_comments=3,  skip_first=2):
    subreddit = reddit.subreddit(subreddit)
    
    titles_and_comments = {}
    
    for counter, post in enumerate(subreddit.hot(limit=limit)):
        
        if counter < skip_first:
            continue
            
        counter += (1-skip_first)
        
        titles_and_comments[counter] = ""
        submission = reddit.submission(post.id)
        
        title = post.title
        
        titles_and_comments[counter] += "Title: " + title + "\n\n"
        titles_and_comments[counter] += "Comments: \n\n"
        
        comment_counter = 0
        
        for comment in submission.comments:
            if not comment.body == "[deleted]":
                
                titles_and_comments[counter] += comment.body+"/n"
                comment_counter +=1
            
            if comment_counter == num_comments:
                break
                
    return titles_and_comments    

In [31]:
titles_and_comments = get_titles_and_comments()

In [37]:
def create_prompt(titles_and_comments):
    task = "Return the stock ticker or company name mentioned in the following title and comments and classify the sentiment around the company as positive, negative or neutral. If not ticker or company is mentioned write 'No company mentioned'.\n\n"
    return task+titles_and_comments

In [38]:
print(create_prompt(titles_and_comments[1]))

Return the stock ticker or company name mentioned in the following title and comments and classify the sentiment around the company as positive, negative or neutral. If not ticker or company is mentioned write 'No company mentioned'.

Title: Why is the market pricing in rate cutes when the fed has been clear they arent thinking about cuts?

Comments: 

They saw JPOW wink when he said no rate cuts./nOf course the Fed is going to say that. Inflation is partly psychological so if they let out that they will lower rates as of X date, it makes the Fed's job much harder and the chances of a soft landing gone.

The Fed's playbook has always been, "do whatever it takes to fight inflation, blah, blah and then lower rates at the first sign of a recession./nHe has to act hawkish to keep markets in control but he damn well knows he's on borrowed time leaving rates where they are at. Markets are attempting to front run this./n


In [44]:
for key, title_with_comments in titles_and_comments.items():
    
    prompt = create_prompt(title_with_comments)
        
    response = openai.Completion.create(
            engine = 'text-davinci-003',
            prompt = prompt,
            max_tokens = 256,
            temperature = 0,
            top_p=1.0
    )
    
    
    print(title_with_comments)
    print(f"Sentiment report from OpenAI {response['choices'][0]['text']}")
    print('--------------------------------------------')

Title: Why is the market pricing in rate cutes when the fed has been clear they arent thinking about cuts?

Comments: 

They saw JPOW wink when he said no rate cuts./nOf course the Fed is going to say that. Inflation is partly psychological so if they let out that they will lower rates as of X date, it makes the Fed's job much harder and the chances of a soft landing gone.

The Fed's playbook has always been, "do whatever it takes to fight inflation, blah, blah and then lower rates at the first sign of a recession./nHe has to act hawkish to keep markets in control but he damn well knows he's on borrowed time leaving rates where they are at. Markets are attempting to front run this./n
Sentiment report from OpenAI 

No company mentioned.
--------------------------------------------
Title: Apple TV Plus and Paramount Plus reportedly discussing cheap bundle plan to take on Netflix

Comments: 

Apple TV+ is a high quality product and I just don’t see the same appeal for paramount./nThe slow